DOWNLOADING THE YOUTUBE VIDEO

In [1]:
! pip install pytube -q 

     |████████████████████████████████| 56 kB 2.3 MB/s 


In [2]:
from pytube import YouTube

In [3]:
VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ' #batman

In [4]:
vid = YouTube(VIDEO_URL)

In [5]:
vid.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'audio.mp4') \

'/content/audio.mp4'

MP4 TO WAV

In [6]:
! ffmpeg -i audio.mp4 -acodec pcm_s16le -ar 16000 audio.wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

IMPORTING THE HUGGINGSOUND MODEL

In [7]:
!pip install huggingsound -q 

     |████████████████████████████████| 312 kB 7.7 MB/s 
     |████████████████████████████████| 25.3 MB 57.5 MB/s 
     |████████████████████████████████| 3.4 MB 44.6 MB/s 
     |████████████████████████████████| 4.2 MB 35.7 MB/s 
     |████████████████████████████████| 212 kB 47.5 MB/s 
     |████████████████████████████████| 136 kB 49.9 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 127 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.3 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 271 kB 54.7 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 50.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [8]:
from huggingsound import SpeechRecognitionModel

In [9]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda'

In [11]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

05/14/2022 10:40:16 - INFO - huggingsound.speech_recognition.model - Loading model...


Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

TO AVOID OUT OF MEMORY ERROR WE WILL PROCESS THE AUDIO FILE IN CHUNKS

In [12]:
import librosa

In [13]:
input_file = '/content/audio.wav'

In [14]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [15]:
import soundfile as sf

In [16]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [22]:
i+1 #displayes the number of chunks made

9

AUDIO TRANSCRIPTION

In [19]:
audio_path =[] #appending all chunks in one list
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [20]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav']

In [21]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 9/9 [00:10<00:00,  1.17s/it]


In [23]:
full_transcript = ' '

In [24]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [25]:
len(full_transcript)

3091

In [26]:
full_transcript

" someone has been very vocal about being a batman then for prety muture whole lifeh described the filling a putt on the batsy for the first time waempowering emotionalyfrom getting costs to putting it on the first timeproly months so you gone through holting with a  reactionyour own reactioreactio eventryprepare foritmreading the most untold number of graphic novelsandwhen you findlly get to put on the suit its just suddenind of itmakes it real but isuddenyyou suddenly think you can dosomething totally different withte character its wed thatyouput it heistthat you put it on the first amnual andits so well-designed i fit so perfitlyan you ca movein it so well andyou just looki theycied powerful n you can really that so much history is imbutsuit andcnography y feel it when you put it onthothe onscrean the role of cat woman has been played by litedary access arears from rto kit and shell fifer how does it fal to take on the mantle of such a powerful female characterits scarry its scarry 

TEXT SUMMARIZATION

In [27]:
from transformers import pipeline

In [28]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [29]:
summarized_text = summarization(full_transcript)

In [30]:
summarized_text[0]['summary_text']

" The role of cat woman has been played by litedary access arears from rto kit and shell fifer . Batman is about etwo sides of drauma batman's born of trama andtis film maybes the ridler and tat is kind of the seed from which everything else grew ."